## A. Upload CSVs to S3 (using boto3)

In [ ]:
import boto3
import os

# Setup
bucket_name = "amdari-etl-bucket"
folder_prefix = "medoptix/raw/"
local_folder = "./medoptix_data"

# AWS credentials assumed to be in ~/.aws/credentials or env vars
s3 = boto3.client("s3")

# Upload all CSVs in folder with ACL: bucket-owner-full-control
for file_name in os.listdir(local_folder):
    if file_name.endswith(".csv"):
        file_path = os.path.join(local_folder, file_name)
        s3.upload_file(
            Filename=file_path,
            Bucket=bucket_name,
            Key=folder_prefix + file_name, 
            ExtraArgs={'ACL': 'bucket-owner-full-control'}  # <-- THIS IS KEY
        )
        print(f"✅ Uploaded {file_name} to s3://{bucket_name}/{folder_prefix}{file_name} with full access")


In [ ]:
# !pip install sqlalchemy
# !pip install sqlalchemy psycopg2-binary pandas


## B. Load from S3 → PostgreSQL


In [1]:
import pandas as pd
from sqlalchemy import create_engine
import boto3

s3 = boto3.client("s3")


bucket = "amdari-etl-bucket"
prefix = "medoptix/raw/"

# Files to download
files = ["patients.csv", "clinics.csv", "sessions.csv", "feedback.csv", "dropout_flags.csv"]

# Download files
for file in files:
    s3.download_file(bucket, prefix + file, file)
    print(f"⬇️ Downloaded {file} from S3")



⬇️ Downloaded patients.csv from S3
⬇️ Downloaded clinics.csv from S3
⬇️ Downloaded sessions.csv from S3
⬇️ Downloaded feedback.csv from S3
⬇️ Downloaded dropout_flags.csv from S3


patients (PK: patient_id)
 └──< sessions (PK: session_id, FK: patient_id)
         └──< feedback (PK: feedback_id, FK: session_id)


In [3]:
import pandas as pd
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv

# Load environment variables (recommended for security)
load_dotenv()

def get_db_engine():
    """Create and return a SQLAlchemy engine with proper connection string"""
    # Construct connection string from environment variables
    db_url = (
        f"postgresql://{'avnadmin'}:{'REDACTED'}@"
        f"{'pg-3c2ee52c-muhammadnurudeen-e10b.g.aivencloud.com'}:{'11234'}/{'defaultdb'}?"
    )
    return create_engine(db_url)

def upload_data():
    # Step 1: Read CSV files
    patients = pd.read_csv("C:/Users/Muham/Downloads/Medoptix_Demo/medoptix_data/processed/patients.csv")
    sessions = pd.read_csv("C:/Users/Muham/Downloads/Medoptix_Demo/medoptix_data/processed/sessions.csv")
    feedback = pd.read_csv("C:/Users/Muham/Downloads/Medoptix_Demo/medoptix_data/processed/feedback.csv")

    # Step 2: Create database engine
    engine = get_db_engine()

    # Step 3: Upload in referential order with error handling
    with engine.begin() as connection:  # Automatically handles transactions
        # Chunk size for large datasets (adjust as needed)
        chunk_size = 1000

        # Upload patients table
        patients.to_sql(
            "patients", 
            connection, 
            if_exists="append", 
            index=False,
            chunksize=chunk_size,
            method='multi'  # Faster for bulk inserts
        )

        # Upload sessions table
        sessions.to_sql(
            "sessions", 
            connection, 
            if_exists="append", 
            index=False,
            chunksize=chunk_size,
            method='multi'
        )

        # Upload feedback table
        feedback.to_sql(
            "feedback", 
            connection, 
            if_exists="append", 
            index=False,
            chunksize=chunk_size,
            method='multi'
        )

    print("✅ Data uploaded successfully with relationships intact.")

if __name__ == "__main__":
    upload_data()


✅ Data uploaded successfully with relationships intact.


In [6]:
from sqlalchemy import create_engine
import pandas as pd


def get_db_engine():
    """Create and return a SQLAlchemy engine with proper connection string"""
    # Construct connection string from environment variables
    db_url = (
        f"postgresql://{'avnadmin'}:{'REDACTED'}@"
        f"{'pg-3c2ee52c-muhammadnurudeen-e10b.g.aivencloud.com'}:{'11234'}/{'defaultdb'}?"
    )
    return create_engine(db_url)


engine = get_db_engine()


# Query to fetch data from the tables
patients_query = "SELECT * FROM patients"
sessions_query = "SELECT * FROM sessions"
feedback_query = "SELECT * FROM feedback"

# Load data into pandas DataFrame
patients_df = pd.read_sql(patients_query, engine)
sessions_df = pd.read_sql(sessions_query, engine)
feedback_df = pd.read_sql(feedback_query, engine)




In [8]:
# Display basic info and summary statistics for each table

# Patients
print("Patients Data Overview")
patients_df.head()
patients_df.info()
patients_df.describe()

Patients Data Overview
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   patient_id       5000 non-null   int64         
 1   age              5000 non-null   int64         
 2   gender           5000 non-null   object        
 3   bmi              5000 non-null   float64       
 4   smoker           5000 non-null   bool          
 5   chronic_cond     1931 non-null   object        
 6   injury_type      5000 non-null   object        
 7   signup_date      5000 non-null   datetime64[ns]
 8   referral_source  5000 non-null   object        
 9   consent          5000 non-null   bool          
 10  clinic_id        5000 non-null   int64         
 11  insurance_type   5000 non-null   object        
dtypes: bool(2), datetime64[ns](1), float64(1), int64(3), object(5)
memory usage: 400.5+ KB


,patient_id,age,bmi,signup_date,clinic_id
count,5000.000000,5000.000000,5000.000000,5000,5000.000000
mean,2500.500000,50.907200,25.174580,2024-12-10 09:48:44.286283520,3.387800
min,1.000000,18.000000,17.000000,2024-06-13 00:12:44.286283,1.000000
25%,1250.750000,34.000000,22.400000,2024-09-08 00:12:44.286283008,2.000000
50%,2500.500000,51.000000,25.100000,2024-12-10 12:12:44.286283008,3.000000
75%,3750.250000,68.000000,27.700000,2025-03-11 00:12:44.286283008,5.000000
max,5000.000000,84.000000,38.800000,2025-06-12 00:12:44.286283,8.000000
std,1443.520003,19.403203,3.769633,NaN,1.749521


In [9]:
# Sessions
print("Sessions Data Overview")
sessions_df.head()
sessions_df.info()
sessions_df.describe()

Sessions Data Overview
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70236 entries, 0 to 70235
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   session_id         70236 non-null  object        
 1   patient_id         70236 non-null  int64         
 2   date               70236 non-null  datetime64[ns]
 3   week               70236 non-null  int64         
 4   duration           70236 non-null  int64         
 5   pain_level         70236 non-null  int64         
 6   exercise_type      70236 non-null  object        
 7   home_adherence_pc  70236 non-null  int64         
 8   satisfaction       70236 non-null  int64         
 9   therapist_id       70236 non-null  int64         
dtypes: datetime64[ns](1), int64(7), object(2)
memory usage: 5.4+ MB


,patient_id,date,week,duration,pain_level,home_adherence_pc,satisfaction,therapist_id
count,70236.000000,70236,70236.000000,70236.000000,70236.000000,70236.000000,70236.000000,70236.000000
mean,2498.827652,2025-01-20 12:53:29.801404160,6.490717,39.511134,1.982317,69.333775,6.490831,1099.411498
min,1.000000,2024-06-13 00:12:44.286283,1.000000,20.000000,1.000000,0.000000,2.000000,1000.000000
25%,1247.000000,2024-10-20 00:12:44.286283008,3.000000,30.000000,1.000000,59.000000,6.000000,1049.000000
50%,2499.000000,2025-01-21 00:12:44.286283008,6.000000,40.000000,1.000000,69.000000,6.000000,1099.000000
75%,3751.000000,2025-04-21 00:12:44.286283008,9.000000,49.000000,3.000000,80.000000,7.000000,1149.000000
max,5000.000000,2025-09-02 00:12:44.286283,12.000000,59.000000,8.000000,100.000000,10.000000,1199.000000
std,1444.925172,NaN,3.451124,11.516467,1.484108,14.846721,1.236411,57.610261


In [10]:
# Feedback
print("Feedback Data Overview")
feedback_df.head()
feedback_df.info()
feedback_df.describe()

Feedback Data Overview
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49165 entries, 0 to 49164
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   feedback_id  49165 non-null  object 
 1   session_id   49165 non-null  object 
 2   comments     49165 non-null  object 
 3   sentiment    49165 non-null  float64
dtypes: float64(1), object(3)
memory usage: 1.5+ MB


,sentiment
count,49165.000000
mean,0.304360
std,0.356693
min,-0.440000
25%,0.000000
50%,0.459000
75%,0.542000
max,0.802000


# EDA

### Data Cleaning (Handling Missing Values)

In [19]:
#5. Check for Missing Values in All Tables
print("\nMissing Values in Patients Data:")
print(patients_df.isnull().sum())

print("\nMissing Values in Sessions Data:")
print(sessions_df.isnull().sum())

print("\nMissing Values in Feedback Data:")
print(feedback_df.isnull().sum())

# 6. Clean Missing Values
# Example of filling missing values for categorical and numerical data
patients_df['chronic_cond'] = patients_df['chronic_cond'].fillna(patients_df['chronic_cond'].mode()[0])


Missing Values in Patients Data:
patient_id            0
age                   0
gender                0
bmi                   0
smoker                0
chronic_cond       3069
injury_type           0
signup_date           0
referral_source       0
consent               0
clinic_id             0
insurance_type        0
dtype: int64

Missing Values in Sessions Data:
session_id           0
patient_id           0
date                 0
week                 0
duration             0
pain_level           0
exercise_type        0
home_adherence_pc    0
satisfaction         0
therapist_id         0
dtype: int64

Missing Values in Feedback Data:
feedback_id    0
session_id     0
comments       0
sentiment      0
dtype: int64


In [23]:
print(patients_df.isnull().sum())

patient_id         0
age                0
gender             0
bmi                0
smoker             0
chronic_cond       0
injury_type        0
signup_date        0
referral_source    0
consent            0
clinic_id          0
insurance_type     0
dtype: int64


In [24]:
# 7. Convert Data Types
# Ensure correct data types for certain columns (e.g., 'signup_date' should be datetime)
patients_df['signup_date'] = pd.to_datetime(patients_df['signup_date'])
sessions_df['date'] = pd.to_datetime(sessions_df['date'])


### Data transformation (Feature Engineering)

In [26]:
import numpy as np
# 8. Data Transformation: Feature Engineering (If Necessary)
# Example: Add age group column for segmentation
bins = [0, 18, 35, 55, np.inf]
labels = ['0-18', '19-35', '36-55', '55+']
patients_df['age_group'] = pd.cut(patients_df['age'], bins=bins, labels=labels)

In [28]:
patients_df.head(2)

,patient_id,age,gender,bmi,smoker,chronic_cond,injury_type,signup_date,referral_source,consent,clinic_id,insurance_type,age_group
0,1,29,Male,23.9,False,Diabetes,Knee,2025-04-27 00:12:44.286283,Insurance,True,6,Public,19-35
1,2,68,Female,31.7,True,Diabetes,Shoulder,2024-11-28 00:12:44.286283,Insurance,False,3,Public,55+


### Data Distribution 

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Histogram for distribution of pain level
sns.histplot(sessions_df['pain_level'], kde=True)
plt.title('Distribution of Pain Level')
plt.show()

# Age distribution for patients (histogram)
sns.histplot(patients_df['age'], kde=True)
plt.title('Age Distribution')
plt.show()






